**1. Imports and Helper Functions**

In [2]:
import cv2  # OpenCV library for image processing
import os  # OS module for interacting with the operating system

# Lambda function to split a byte into three parts: 
# - First 3 bits (MSB), next 3 bits, and last 2 bits (LSB)
split_byte_to_bits = lambda data: [data >> 5, (data >> 2) & 0x7, data & 0x3]

# Lambda function to extract the least significant n bits from a byte
extract_nbits_of_byte = lambda band, n: band & (2**n - 1)

# Lambda function to merge 3 bit segments (red, green, and blue bits) into a single byte
merge_bits = lambda rbits, gbits, bbits: (((rbits << 3) | gbits) << 2) | bbits

**2. Generate Embedded Image Name**

In [4]:
# Function to generate the name of an embedded image file
# It replaces the filename in the given path with a prefixed "e_" and converts the extension to .png
def generate_embedded_imagename(vessel_img):
    if '/' in vessel_img:  # Check if the path uses '/' as the separator
        temp = vessel_img.split('/')  # Split the path into components
        temp[-1] = 'e_' + temp[-1]  # Add the "e_" prefix to the last part (filename)
        ename = '/'.join(temp)  # Join the components back into a path
        
        # Replace .jpg or .jpeg extension with .png
        if ename.lower().endswith('.jpg'):
            ename = ename.replace('.jpg', '.png')
        elif ename.lower().endswith('.jpeg'):
            ename = ename.replace('.jpeg', '.png')
        
        return ename  # Return the modified file path
    else:
        # If the path does not use '/', prompt the user to use '/' as the separator
        print('Use / as separator')
        return Non

**3. Generate Header for Embedding(For standardized header)**

In [6]:
def generate_header(doc):
    # This function generates a 30-byte header string for a document.
    # The header includes:
    # - The file name, which is 20 bytes (trimmed or padded with '*')
    # - The file size, which is 10 bytes (right-aligned and padded with '*')

    # Extract the file name from the provided path
    # E.g., for '/Users/sakshizanjad/Downloads/ADAP/Stegano file.rtf', it extracts 'Stegano file.rtf'
    name = doc.split('/')[-1]  # Split the path by '/' and take the last part (file name)

    l = len(name)  # Get the length of the file name
    if l > 20:
        name = name[l - 20:]  # Trim the file name to the last 20 characters if it's too long
    elif l < 20:
        name = name.rjust(20, '*')  # Pad the file name on the left with '*' to make it 20 characters

    # Get the file size in bytes
    size = str(os.path.getsize(doc))  # Convert the file size to a string

    # Ensure the file size is 10 characters, right-aligned and padded with '*'
    size = size.rjust(10, '*')

    # Return the concatenated header (20 bytes for the name + 10 bytes for the size)
    return name + size


**4. Embed Data into Image**

In [8]:
def embed(vessel_image, doc):
    # Embeds the contents of a document into an image for steganography.

    # Check if the vessel image exists
    if not os.path.exists(vessel_image):
        print(vessel_image, 'not found')
        return None

    # Check if the document exists
    if not os.path.exists(doc):
        print(doc, 'not found')
        return None

    # Load the image into memory as a NumPy array (type: numpy.ndarray)
    # mem_image.shape --> (height, width, color_channels)
    mem_image = cv2.imread(vessel_image, cv2.IMREAD_COLOR)
    h, w, _ = mem_image.shape  # Extract height (h), width (w), and channel size (_)

    # Get the size of the document in bytes
    doc_size = os.path.getsize(doc)

    # Generate the header (metadata for the embedded document)
    header = generate_header(doc)

    # Calculate the embedding capacity (total number of pixels in the image)
    capacity = h * w
    header_length = len(header)  # Header length (fixed at 30 bytes)

    # Check if the document (including the header) can fit in the image
    if doc_size + header_length > capacity:
        print(doc, 'too large to fit in', vessel_image)
        return None

    # Start embedding the data
    cnt = 0  # Byte counter for tracking progress
    file_handle = open(doc, 'rb')  # Open the document in binary read mode

    flag = True  # To stop embedding when EOF is reached
    i = 0  # Row index
    while i < h and flag:  # Iterate over rows
        j = 0  # Column index
        while j < w and flag:  # Iterate over columns in the current row
            # Fetch the current pixel's color components (B, G, R)
            pixel = mem_image[i, j]
            blue = pixel[0]
            green = pixel[1]
            red = pixel[2]

            # Determine the source of the data byte to embed
            if cnt < header_length:
                # Embed from the header
                data = ord(header[cnt])  # Convert header character to its ASCII value
            else:
                # Embed from the document
                data = file_handle.read(1)  # Read a byte from the file
                if data:  # If data is valid (not EOF)
                    data = int.from_bytes(data, byteorder='big')  # Convert byte to integer
                else:
                    # Reached EOF
                    flag = False  # Stop embedding
                    continue

            # Split the data byte into three segments: 3 bits, 3 bits, 2 bits
            bits = split_byte_to_bits(data)

            # Embed the bits into the pixel's color channels
            red = (red & (~0x7)) | bits[0]  # Embed the first 3 bits into the red channel
            green = (green & (~0x7)) | bits[1]  # Embed the next 3 bits into the green channel
            blue = (blue & (~0x3)) | bits[2]  # Embed the last 2 bits into the blue channel

            # Update the pixel in the memory image
            mem_image[i, j, 0] = blue
            mem_image[i, j, 1] = green
            mem_image[i, j, 2] = red

            cnt += 1  # Move to the next byte
            j += 1  # Move to the next column
        i += 1  # Move to the next row

    file_handle.close()  # Close the document file

    # Save the modified image with an embedded file
    target_image = generate_embedded_imagename(vessel_image)  # Generate a new image name
    cv2.imwrite(target_image, mem_image)  # Save the embedded image

    return target_image  # Return the name of the saved embedded image


**5. Extract Data from Image**

In [10]:
def extract(embedded_image, target_folder):
    # Extracts the embedded document from an image and saves it to a specified folder.

    # Check if the embedded image exists
    if not os.path.exists(embedded_image):
        print(embedded_image, 'not found')
        return None

    # Check if the target folder exists
    if not os.path.exists(target_folder):
        print(target_folder, 'not found')
        return None

    # Load the embedded image into memory as a NumPy array
    mem_img = cv2.imread(embedded_image, cv2.IMREAD_COLOR)

    # Fetch the size of the image (height, width, and color channels)
    h, w, _ = mem_img.shape

    # Fixed header length (20 bytes for filename + 10 bytes for file size)
    header_length = 30
    flag = True  # Flag to indicate when to stop extraction
    i = 0  # Row index
    cnt = 0  # Byte counter for tracking progress
    header = ''  # Variable to store the extracted header

    # Loop through the pixels to extract the data
    while i < h and flag:  # Iterate over rows
        j = 0  # Column index
        while j < w and flag:  # Iterate over columns in the current row
            # Fetch the current pixel's color components (Blue, Green, Red)
            pixel = mem_img[i, j]
            blue = pixel[0]
            green = pixel[1]
            red = pixel[2]

            # Extract the embedded bits from the pixel's color channels
            red_bits = extract_nbits_of_byte(red, 3)  # Extract 3 bits from the red channel
            green_bits = extract_nbits_of_byte(green, 3)  # Extract 3 bits from the green channel
            blue_bits = extract_nbits_of_byte(blue, 2)  # Extract 2 bits from the blue channel

            # Merge the extracted bits to reconstruct the original byte
            data = merge_bits(red_bits, green_bits, blue_bits)

            if cnt < header_length:
                # Extract header data (filename and file size)
                header = header + chr(data)  # Convert the byte to a character and append
            else:
                if cnt == header_length:
                    # Process the header after it's fully extracted
                    print(header)  # Debug: Print the header
                    filename = header[:20].strip('*')  # Extract and clean the filename
                    filesize = int(header[20:].strip('*'))  # Extract and convert file size

                    # Open the target file for writing in binary mode
                    file_handle = open(target_folder + '/' + filename, 'wb')

                if cnt - header_length < filesize:
                    # Extract the file data if still within the file size limit
                    # Convert NumPy integer to Python integer, then to a byte object
                    data = int.to_bytes(int(data), 1, byteorder='big')
                    file_handle.write(data)  # Write the byte object to the file
                else:
                    # File extraction complete
                    file_handle.close()  # Close the file
                    flag = False  # Stop extraction

            cnt += 1  # Increment the byte counter
            j += 1  # Move to the next column
        i += 1  # Move to the next row

    # Return the path of the extracted file
    return target_folder + '/' + filename

**6. Main Function: Menu for User Interaction**

In [ ]:
def main():
    # Main function to provide a menu-driven interface for embedding and extracting data.

    while True:
        # Display the menu options
        print('1. Click here to EMBED')
        print('2. Click here to EXTRACT')
        print('3. EXIT')
        print('Enter Your Choice: ')
        ch = int(input())  # Get the user's choice

        if ch == 1:  # Option 1: Embed a file into an image
            print('Enter vessel image path')  # Prompt for the image file
            vessel_img = input()
            print('Enter file to embed')  # Prompt for the file to embed
            doc = input()

            # Call the embed function
            result = embed(vessel_img, doc)
            if result is not None:  # Check if embedding was successful
                print('Embedding Done, result: ', result)  # Display success message
            else:
                print('Embedding Failed')  # Display failure message

        elif ch == 2:  # Option 2: Extract a file from an embedded image
            print('Enter embedded image path')  # Prompt for the embedded image file
            embedded_image = input()
            print('Enter target folder for saving the extracted file')  # Prompt for target folder
            target_folder = input()

            # Call the extract function
            result = extract(embedded_image, target_folder)
            if result is not None:  # Check if extraction was successful
                print('Extraction Done, result: ', result)  # Display success message
            else:
                print('Extraction Failed')  # Display failure message

        elif ch == 3:  # Option 3: Exit the program
            break  # Exit the loop and terminate the program

        else:  # Invalid option
            print('Wrong Choice')  # Notify the user of an invalid choice

# Call the main function to start the program
main()

1. Click here to EMBED
2. Click here to EXTRACT
3. EXIT
Enter Your Choice: 


 1


Enter vessel image path


 /Users/sakshizanjad/Downloads/ADAP/images/image5.png


Enter file to embed


...

Enter embedded image path


 /Users/sakshizanjad/Downloads/ADAP/images/e_image5.png


Enter target folder for saving the extracted file


 /Users/sakshizanjad/Downloads/ADAP/Extracted text files


****Stegano file.rtf*******414
Extraction Done, result:  /Users/sakshizanjad/Downloads/ADAP/Extracted text files/Stegano file.rtf
1. Click here to EMBED
2. Click here to EXTRACT
3. EXIT
Enter Your Choice: 
